# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:

city_weather_df = pd.read_csv('../Weather/City_Weather_Data.csv')
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ramygala,55.5167,24.3000,50.00,87,75,5.75,LT,1619757685
1,ushuaia,-54.8000,-68.3000,39.20,75,90,5.75,AR,1619757685
2,medicine hat,50.0501,-110.6683,59.00,41,75,9.22,CA,1619757605
3,ocampo,21.6500,-101.5000,63.32,40,29,7.63,MX,1619757686
4,avera,33.1940,-82.5271,71.60,68,1,9.22,US,1619757609
...,...,...,...,...,...,...,...,...,...
547,victoria point,-27.5833,153.3000,68.00,88,82,1.99,AU,1619757934
548,raahe,64.6833,24.4833,33.01,100,20,6.91,FI,1619757935
549,galdar,28.1470,-15.6502,63.00,82,20,2.30,ES,1619757935
550,gore,-46.1028,168.9436,53.01,72,39,10.00,NZ,1619757935


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Get lat and lng of cities
cities = [(city_weather_df.iloc[i, 1], city_weather_df.iloc[i, 2]) for i in city_weather_df.index]
gmaps.configure(api_key=g_key)
fig = gmaps.figure(map_type='ROADMAP', zoom_level=2, center=(0,0))
heatmap = gmaps.heatmap_layer(cities, weights=city_weather_df['Humidity'])
fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
ideal_weather =[]

for i in city_weather_df.index:
    if (city_weather_df.iloc[i, 3] > 65 and city_weather_df.iloc[i, 3] < 85) and (city_weather_df.iloc[i, 5] <= 25 and city_weather_df.iloc[i, 6] <= 5):
        ideal_weather.append(
            {'City': city_weather_df.iloc[i,0],
             'Lat': city_weather_df.iloc[i,1],
             'Lng': city_weather_df.iloc[i,2],
             'Max Temp': city_weather_df.iloc[i,3],
             'Humidity': city_weather_df.iloc[i,4],
             'Cloudiness': city_weather_df.iloc[i,5],
             'Wind Speed': city_weather_df.iloc[i,6],
             'Country': city_weather_df.iloc[i,7]
             })
ideal_weather_df = pd.DataFrame(ideal_weather)
ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,atuona,-9.8000,-139.0333,79.34,70,8,4.52,PF
1,flinders,-34.5833,150.8552,72.00,72,9,3.00,AU
2,maxixe,-23.8597,35.3472,66.20,94,20,3.98,MZ
3,morgan city,29.6994,-91.2068,75.20,100,1,4.61,US
4,hobart,-42.8794,147.3294,68.00,55,20,3.44,AU
5,beroroha,-21.6667,45.1667,76.86,63,0,4.79,MG
6,thinadhoo,0.5333,72.9333,83.50,70,20,4.45,MV
7,komsomolskiy,40.4272,71.7189,84.20,18,3,4.61,UZ
8,saint george,37.1041,-113.5841,75.20,17,1,4.27,US
9,monrovia,6.3005,-10.7969,77.00,94,20,3.85,LR


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = ideal_weather_df
hotel_df['Hotel Name'] = ''

params = {'key': g_key}

# Use for loop to go through cities and get lat/lng for each one and make requests
for index, row in hotel_df.itterows():
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    city = row['City']
    params['address'] = f'{city}'
    response = requests.get(url, params=params)
    ideal_places = response.json()
    # Set locations of hotels
    hotel_df.loc[index, 'Lat'] = ideal_places['results'][0]['geometry']['location']['lat']
    hotel_df.loc[index, 'Lng'] = ideal_places['results'][0]['geometry']['location']['lng']

# Set up url



In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
